In [10]:
import random
import math

def roll_dice_pool(pool_size):
    rolls = [random.randint(1,10) for _ in range(pool_size)]
    auto_success = pool_size // 2
    bonus_success = 0
    fail_count = 0
    raw_success = 0
    for r in rolls:
        if r == 1:
            fail_count += 1
        elif r == 10:
            if pool_size >= 10:
                bonus_success += 2
            else:
                bonus_success += 1
        elif r >= 6:
            raw_success += 1
    total_success = auto_success + bonus_success + raw_success - fail_count
    return {
        "rolls": rolls,
        "auto_success": auto_success,
        "bonus_success": bonus_success,
        "raw_success": raw_success,
        "fail_count": fail_count,
        "total_success": total_success
    }

class Character:
    def __init__(self, name, stats):
        self.name = name
        self.stats = stats
        self.init_roll = 0
        self.PAP = 0
        self.MAP = 0
        self.TAP = 0
        self.physical_exertion = 0
        self.mental_exertion = 0
        self.tap_exertion = 0
        self.permanent_tap_loss = 0
        self.remaining_PAP = 0
        self.remaining_MAP = 0
        self.remaining_TAP = 0

    def roll_initiative(self):
        pool = self.stats["DEX"] + self.stats["AWA"] + self.stats["SAV"]
        result = roll_dice_pool(pool)
        self.init_roll = result['total_success']
        print(f"{self.name} Initiative dice: {result['rolls']}")
        print(f"  Auto: {result['auto_success']}, Bonus(10s): {result['bonus_success']},"
              f" Success(6+): {result['raw_success']}, Fails(1s): {result['fail_count']} --> Initiative: {self.init_roll}")

    def setup_ap(self):
        self.PAP = self.stats["STR"] + self.stats["DEX"] + self.stats["CON"] + self.init_roll
        self.MAP = self.stats["INT"] + self.stats["AWA"] + self.stats["WILL"] + self.init_roll
        self.TAP = self.PAP + self.MAP
        self.remaining_PAP = self.PAP
        self.remaining_MAP = self.MAP
        self.remaining_TAP = self.TAP

    def print_ap_calculation(self):
        print(f"{self.name} AP Calculation:")
        pap_str = f"PAP = STR({self.stats['STR']}) + DEX({self.stats['DEX']}) + CON({self.stats['CON']}) + Initiative({self.init_roll}) = {self.stats['STR']} + {self.stats['DEX']} + {self.stats['CON']} + {self.init_roll} = {self.PAP}"
        map_str = f"MAP = INT({self.stats['INT']}) + AWA({self.stats['AWA']}) + WILL({self.stats['WILL']}) + Initiative({self.init_roll}) = {self.stats['INT']} + {self.stats['AWA']} + {self.stats['WILL']} + {self.init_roll} = {self.MAP}"
        tap_str = f"TAP = PAP({self.PAP}) + MAP({self.MAP}) = {self.PAP} + {self.MAP} = {self.TAP}"
        print(pap_str)
        print(map_str)
        print(tap_str)

    def start_turn(self):
        self.PAP = self.remaining_PAP + self.stats["STR"] + self.stats["DEX"] + (2 * self.stats["CON"]) + self.init_roll - self.physical_exertion
        self.MAP = self.remaining_MAP + self.stats["INT"] + self.stats["AWA"] + (2 * self.stats["WILL"]) + self.init_roll - self.mental_exertion
        self.TAP = self.remaining_TAP + self.PAP + self.MAP - self.tap_exertion

    def spend_ap(self, pap=0, map=0):
        self.PAP -= pap
        self.MAP -= map
        self.TAP -= (pap + map)
        self.remaining_PAP = self.PAP
        self.remaining_MAP = self.MAP
        self.remaining_TAP = self.TAP

    def apply_exertion(self, turn_num):
        self.PAP -= turn_num
        self.MAP -= turn_num
        self.TAP -= 2 * turn_num
        self.remaining_PAP = self.PAP
        self.remaining_MAP = self.MAP
        self.remaining_TAP = self.TAP
        tap_spent = self.TAP if self.TAP > 0 else 0
        if tap_spent > 0:
            extra = math.ceil(tap_spent / 10)
            self.TAP -= extra
            self.tap_exertion += extra
        pap_exertion = math.ceil(abs(self.PAP) / 10)
        map_exertion = math.ceil(abs(self.MAP) / 10)
        self.physical_exertion += pap_exertion
        self.mental_exertion += map_exertion
        if self.TAP < 0:
            self.permanent_tap_loss += abs(self.TAP)
            self.TAP = 0

    def __repr__(self):
        return (f"{self.name}: PAP={self.PAP}, MAP={self.MAP}, TAP={self.TAP}, "
                f"PhysExert={self.physical_exertion}, MentExert={self.mental_exertion}, "
                f"TAPExert={self.tap_exertion}, PermTAPLoss={self.permanent_tap_loss}")

def print_status(characters, round_desc):
    print(f"\n=== {round_desc} ===")
    for c in characters:
        print(c)

# Sample characters
char1 = Character("Warrior", {"STR":4, "DEX":3, "CON":4, "INT":2, "AWA":3, "WILL":2, "CHA":1, "SAV":2, "COM":2})
char2 = Character("Mage",    {"STR":2, "DEX":3, "CON":2, "INT":5, "AWA":4, "WILL":5, "CHA":2, "SAV":3, "COM":2})
char3 = Character("Rogue",   {"STR":2, "DEX":5, "CON":3, "INT":3, "AWA":4, "WILL":3, "CHA":2, "SAV":4, "COM":3})

characters = [char1, char2, char3]

# Roll initiatives and setup APs
for c in characters:
    c.roll_initiative()
    c.setup_ap()
    c.print_ap_calculation()


# Sort by TAP
characters.sort(key=lambda c: c.TAP, reverse=True)
print_status(characters, "Initial State (after Initiative and AP setup)")

# --- Turn 1 (sample actions) ---
turn = 1
actions = [
    ("attack", "STR", 3, 0, 2),
    ("cast",   "INT", 0, 3, 2),
    ("move",   "DEX", 3, 0, 2),
]
for i, c in enumerate(characters):
    action, attr, pap, map, dc = actions[i]
    print(f"{c.name} attempts action: {action} (cost: {pap} PAP, {map} MAP) - rolling {attr}")
    roll_result = roll_dice_pool(c.stats[attr])
    print(f"  Dice: {roll_result['rolls']}")
    print(f"  Automatic successes: {roll_result['auto_success']}")
    print(f"  Bonus (10s): {roll_result['bonus_success']}, Successes (6+): {roll_result['raw_success']}, Fails (1s): {roll_result['fail_count']}")
    print(f"  => Total Successes: {roll_result['total_success']} vs DC {dc} --> {'SUCCESS' if roll_result['total_success'] >= dc else 'FAIL'}")
    c.spend_ap(pap, map)
print_status(characters, "After Actions (Turn 1)")

# End of turn exertion
for c in characters:
    c.apply_exertion(turn)
print_status(characters, "After Exertion (Turn 1)")


Warrior Initiative dice: [2, 3, 7, 8, 2, 10, 10, 7]
  Auto: 4, Bonus(10s): 2, Success(6+): 3, Fails(1s): 0 --> Initiative: 9
Warrior AP Calculation:
PAP = STR(4) + DEX(3) + CON(4) + Initiative(9) = 4 + 3 + 4 + 9 = 20
MAP = INT(2) + AWA(3) + WILL(2) + Initiative(9) = 2 + 3 + 2 + 9 = 16
TAP = PAP(20) + MAP(16) = 20 + 16 = 36
Mage Initiative dice: [6, 7, 3, 7, 7, 5, 5, 5, 10, 1]
  Auto: 5, Bonus(10s): 2, Success(6+): 4, Fails(1s): 1 --> Initiative: 10
Mage AP Calculation:
PAP = STR(2) + DEX(3) + CON(2) + Initiative(10) = 2 + 3 + 2 + 10 = 17
MAP = INT(5) + AWA(4) + WILL(5) + Initiative(10) = 5 + 4 + 5 + 10 = 24
TAP = PAP(17) + MAP(24) = 17 + 24 = 41
Rogue Initiative dice: [9, 3, 3, 7, 3, 10, 6, 1, 1, 4, 7, 1, 9]
  Auto: 6, Bonus(10s): 2, Success(6+): 5, Fails(1s): 3 --> Initiative: 10
Rogue AP Calculation:
PAP = STR(2) + DEX(5) + CON(3) + Initiative(10) = 2 + 5 + 3 + 10 = 20
MAP = INT(3) + AWA(4) + WILL(3) + Initiative(10) = 3 + 4 + 3 + 10 = 20
TAP = PAP(20) + MAP(20) = 20 + 20 = 40

=== I

In [ ]:
import random
import math

def roll_dice_pool(pool_size):
    rolls = [random.randint(1,10) for _ in range(pool_size)]
    auto_success = pool_size // 2
    bonus_success = 0
    fail_count = 0
    raw_success = 0
    for r in rolls:
        if r == 1:
            fail_count += 1
        elif r == 10:
            if pool_size >= 10:
                bonus_success += 2
            else:
                bonus_success += 1
        elif r >= 6:
            raw_success += 1
    total_success = auto_success + bonus_success + raw_success - fail_count
    return total_success

class Character:
    def __init__(self, name, stats):
        self.name = name
        self.stats = stats
        self.init_roll = 0
        self.PAP = 0
        self.MAP = 0
        self.TAP = 0
        self.physical_exertion = 0
        self.mental_exertion = 0
        self.tap_exertion = 0
        self.permanent_tap_loss = 0
        self.permanent_pap_loss = 0
        self.permanent_map_loss = 0
        self.remaining_PAP = 0
        self.remaining_MAP = 0
        self.remaining_TAP = 0
        self.turn_pap_spent = 0
        self.turn_map_spent = 0
        self.turn_tap_spent = 0


    def print_total_stats(self):
        total = sum(self.stats.values())
        print(f"{self.name} total attribute points: {total}")

    def roll_initiative(self):
        pool = self.stats["DEX"] + self.stats["AWA"] + self.stats["SAV"]
        rolls = [random.randint(1,10) for _ in range(pool)]
        auto = pool // 2
        bonus = sum(2 if pool >= 10 else 1 for r in rolls if r == 10)
        raw = sum(1 for r in rolls if r >= 6 and r != 10)
        fails = sum(1 for r in rolls if r == 1)
        total = auto + bonus + raw - fails
        print(f"{self.name} Initiative dice: {rolls}")
        print(f"  Auto: {auto}, Bonus(10s): {bonus}, Success(6+): {raw}, Fails(1s): {fails} --> Initiative: {total}")
        self.init_roll = total

    def setup_ap(self):
        self.PAP = self.stats["STR"] + self.stats["DEX"] + self.stats["CON"] + self.init_roll
        self.MAP = self.stats["INT"] + self.stats["AWA"] + self.stats["WILL"] + self.init_roll
        self.TAP = self.PAP + self.MAP
        self.remaining_PAP = self.PAP
        self.remaining_MAP = self.MAP
        self.remaining_TAP = self.TAP

    def print_ap_calculation(self):
        print(f"{self.name} AP Calculation:")
        pap_str = f"PAP = STR({self.stats['STR']}) + DEX({self.stats['DEX']}) + CON({self.stats['CON']}) + Initiative({self.init_roll}) = {self.stats['STR']} + {self.stats['DEX']} + {self.stats['CON']} + {self.init_roll} = {self.PAP}"
        map_str = f"MAP = INT({self.stats['INT']}) + AWA({self.stats['AWA']}) + WILL({self.stats['WILL']}) + Initiative({self.init_roll}) = {self.stats['INT']} + {self.stats['AWA']} + {self.stats['WILL']} + {self.init_roll} = {self.MAP}"
        tap_str = f"TAP = PAP({self.PAP}) + MAP({self.MAP}) = {self.PAP} + {self.MAP} = {self.TAP}"
        print(pap_str)
        print(map_str)
        print(tap_str)    
    
    def start_turn(self):
        self.PAP = (
            self.remaining_PAP
            + self.stats["STR"]
            + self.stats["DEX"]
            + (2 * self.stats["CON"])
            + self.init_roll
            - self.physical_exertion
            - self.permanent_pap_loss
        )
        self.MAP = (
            self.remaining_MAP
            + self.stats["INT"]
            + self.stats["AWA"]
            + (2 * self.stats["WILL"])
            + self.init_roll
            - self.mental_exertion
            - self.permanent_map_loss
        )
        self.TAP = (
            self.remaining_TAP
            + self.PAP
            + self.MAP
            - self.tap_exertion
            - self.permanent_tap_loss
        )

    def spend_ap(self, pap=0, map=0):
        self.PAP -= pap
        self.MAP -= map
        self.TAP -= (pap + map)
        self.remaining_PAP = self.PAP
        self.remaining_MAP = self.MAP
        self.remaining_TAP = self.TAP
        # Track AP spent this turn
        self.turn_pap_spent += pap
        self.turn_map_spent += map
        self.turn_tap_spent += (pap + map)


    def apply_exertion(self, turn_num):
        self.PAP -= turn_num
        self.MAP -= turn_num
        self.TAP -= 2 * turn_num
        self.remaining_PAP = self.PAP
        self.remaining_MAP = self.MAP
        self.remaining_TAP = self.TAP
        tap_spent = self.TAP if self.TAP > 0 else 0
        if tap_spent > 0:
            extra = math.ceil(tap_spent / 10)
            self.TAP -= extra
            self.tap_exertion += extra
        pap_exertion = math.ceil(self.turn_pap_spent / 10)
        map_exertion = math.ceil(self.turn_map_spent / 10)
        tap_exertion = math.ceil(self.turn_tap_spent / 10)
        self.physical_exertion += pap_exertion
        self.mental_exertion += map_exertion
        self.tap_exertion += tap_exertion
        # Reset counters for next turn
        self.turn_pap_spent = 0
        self.turn_map_spent = 0
        self.turn_tap_spent = 0
        
        if self.PAP < 0:
            self.permanent_pap_loss += abs(self.PAP)
            self.PAP = 0
        if self.MAP < 0:
            self.permanent_map_loss += abs(self.MAP)
            self.MAP = 0
        if self.TAP < 0:
            self.permanent_tap_loss += abs(self.TAP)
            self.TAP = 0


    def status(self):
        return f"TAP: {self.TAP}, PAP: {self.PAP}, MAP: {self.MAP}"

    def exertion_report(self):
        return (
            f"{self.name}: PAP={self.PAP}, MAP={self.MAP}, TAP={self.TAP}, "
            f"PhysExert={self.physical_exertion}, MentExert={self.mental_exertion}, "
            f"TAPExert={self.tap_exertion}, "
            f"PermPAPLoss={self.permanent_pap_loss}, PermMAPLoss={self.permanent_map_loss}, PermTAPLoss={self.permanent_tap_loss}"
        )
    
# Characters
char1 = Character("Warrior", {"STR":5, "DEX":5, "CON":5, "INT":2, "AWA":3, "WILL":2, "CHA":1, "SAV":3, "COM":2})
char2 = Character("Mage",    {"STR":2, "DEX":3, "CON":2, "INT":5, "AWA":4, "WILL":5, "CHA":2, "SAV":3, "COM":2})
char3 = Character("Rogue",   {"STR":2, "DEX":5, "CON":3, "INT":3, "AWA":4, "WILL":2, "CHA":2, "SAV":4, "COM":3})

characters = [char1, char2, char3]

# Initiative and AP setup
for c in characters:
    c.print_total_stats()
    c.roll_initiative()
    c.setup_ap()
    c.print_ap_calculation()

# Sort by TAP descending
characters.sort(key=lambda c: c.TAP, reverse=True)

print("=== Initial State (after Initiative and AP setup) ===")
for c in characters:
    print(c.exertion_report())

# Actions for each (for simplicity, same action every turn)
actions = [
    ("attack", 3, 0),   # Warrior uses attack (cost: 3 PAP)
    ("cast",   0, 3),   # Mage uses cast spell (cost: 3 MAP)
    ("move",   3, 0),   # Rogue uses move (cost: 3 PAP)
]

# Simulate 10 turns
for turn in range(1, 11):
    print(f"\n-- Turn {turn} --")
    for idx, c in enumerate(characters):
        act, pap_cost, map_cost = actions[idx]
        c.spend_ap(pap_cost, map_cost)
        print(f"{c.name} attempts {act} (cost: {pap_cost} PAP, {map_cost} MAP) | {c.status()}")
    for c in characters:
        c.apply_exertion(turn)
    print(f"\n=== After Exertion (Turn {turn}) ===")
    for c in characters:
        print(c.exertion_report())



Warrior total attribute points: 28
Warrior Initiative dice: [1, 1, 10, 10, 9, 5, 9, 5, 7, 4, 1]
  Auto: 5, Bonus(10s): 4, Success(6+): 3, Fails(1s): 3 --> Initiative: 9
Warrior AP Calculation:
PAP = STR(5) + DEX(5) + CON(5) + Initiative(9) = 5 + 5 + 5 + 9 = 24
MAP = INT(2) + AWA(3) + WILL(2) + Initiative(9) = 2 + 3 + 2 + 9 = 16
TAP = PAP(24) + MAP(16) = 24 + 16 = 40
Mage total attribute points: 28
Mage Initiative dice: [2, 5, 6, 7, 8, 8, 1, 5, 9, 2]
  Auto: 5, Bonus(10s): 0, Success(6+): 5, Fails(1s): 1 --> Initiative: 9
Mage AP Calculation:
PAP = STR(2) + DEX(3) + CON(2) + Initiative(9) = 2 + 3 + 2 + 9 = 16
MAP = INT(5) + AWA(4) + WILL(5) + Initiative(9) = 5 + 4 + 5 + 9 = 23
TAP = PAP(16) + MAP(23) = 16 + 23 = 39
Rogue total attribute points: 28
Rogue Initiative dice: [4, 5, 8, 8, 5, 10, 5, 10, 6, 2, 4, 3, 4]
  Auto: 6, Bonus(10s): 4, Success(6+): 3, Fails(1s): 0 --> Initiative: 13
Rogue AP Calculation:
PAP = STR(2) + DEX(5) + CON(3) + Initiative(13) = 2 + 5 + 3 + 13 = 23
MAP = INT(3)

: 

In [11]:
import math
import random

# --- 1. ACTIONS (Unchanged) ---
ACTIONS = {
    "Simple Attack": {"cost": 6},
    "Heavy Attack": {"cost": 12},
    "Move": {"cost": 1}
}

# --- 2. CHARACTER CLASS & MECHANICS (Final Cleaned Version) ---
class Character:
    def __init__(self, name: str, str_val: int, dex_val: int, sta_val:int):
        self.name = name
        self.strength = str_val
        self.dexterity = dex_val
        self.stamina = sta_val
        
        self._calculate_derived_stats()

        # Initialize dynamic combat variables
        self.current_ap = 0
        self.current_endurance = 0
        # REMOVED: self.effective_max_ap is no longer needed
        self.initiative_bonus = 0

    def _roll_initiative_successes(self) -> int:
        pool_size = self.strength + self.dexterity + self.stamina
        successes = 0
        for _ in range(pool_size):
            if random.randint(1, 10) >= 6:
                successes += 1
        return successes

    def _calculate_derived_stats(self):
        dex_str_bonus = (self.dexterity * 2) + (self.strength * 1)
        sta_str_bonus = (self.stamina * 2) + (self.strength * 1)

        # RENAMED: base_max_ap is now just max_ap for clarity
        self.max_ap = dex_str_bonus

        self.max_endurance = sta_str_bonus * 3
        self.base_ap_regen = 3 + (dex_str_bonus * (1/3))
        self.endurance_regen = 3 + (sta_str_bonus * (1/3))
        self.endurance_burn_rate = 1

    def initialize_for_combat(self):
        print(f"--- {self.name} enters combat! ---")
        self.display_initial_stats()

        self.initiative_bonus = self._roll_initiative_successes()
        print(f"  Initiative Roll grants: +{self.initiative_bonus} AP (One-time starting bonus)")

        self.current_endurance = self.max_endurance
        
        # Set starting AP using max_ap directly
        self.current_ap = self.max_ap + self.initiative_bonus
        print(f"  Starting AP (Overfilled): {self.current_ap:.2f}")
        print("-" * 20)

    def update_at_round_end(self, ap_spent_this_round: float):
        endurance_lost = ap_spent_this_round * self.endurance_burn_rate
        net_endurance_change = self.endurance_regen - endurance_lost
        self.current_endurance += net_endurance_change
        self.current_endurance = max(0, min(self.max_endurance, self.current_endurance))

        endurance_percentage = self.current_endurance / (self.max_endurance + 1e-6)
        effective_ap_regen = self.base_ap_regen * endurance_percentage
        
        # Use max_ap directly as the cap
        self.current_ap = min(self.max_ap, self.current_ap + effective_ap_regen) 

        print(f"    Net Endurance Change: {net_endurance_change:+.2f} | New Endurance: {self.current_endurance:.2f} ({endurance_percentage:.1%})")
        print(f"    Effective AP Regen this round: {effective_ap_regen:.2f}")
        # REMOVED: The redundant print statement for the cap

    def display_initial_stats(self):
        print(f"  Base Stats: STR {self.strength}, DEX {self.dexterity}, STA {self.stamina}")
        print(f"  Max AP (Permanent Cap): {self.max_ap:.2f}")
        print(f"  Max Endurance: {self.max_endurance:.2f}")
        print(f"  Base AP Regen/Round: {self.base_ap_regen:.2f}")
        print(f"  Endurance Regen/Round: {self.endurance_regen:.2f}")

    def display_round_status(self):
        print(f"  Start of Round Status:")
        # Use max_ap directly for display
        print(f"    AP: {self.current_ap:.2f} / {self.max_ap:.2f} (Current/Cap)")
        print(f"    Endurance: {self.current_endurance:.2f} / {self.max_endurance:.2f}")

# --- 3. SIMULATOR (Unchanged) ---
def aggressive_strategy(character: Character, primary_action: dict):
    ap_spent_this_round = 0
    actions_taken = 0
    while character.current_ap >= primary_action['cost']:
        character.current_ap -= primary_action['cost']
        ap_spent_this_round += primary_action['cost']
        actions_taken += 1
    print(f"  Actions Taken: {actions_taken} x 'Simple Attack' ({ap_spent_this_round} AP spent)")
    return ap_spent_this_round

def simulate_combat(character: Character, num_rounds: int):
    character.initialize_for_combat()
    for i in range(1, num_rounds + 1):
        print(f"\n--- ROUND {i} ---")
        character.display_round_status()
        ap_spent = aggressive_strategy(character, ACTIONS["Simple Attack"])
        character.update_at_round_end(ap_spent)
    print(f"\n--- SIMULATION COMPLETE for {character.name} ---")

# --- 4. RUN THE SIMULATION ---
if __name__ == "__main__":
    print("="*40)
    print("SIMULATION: Final Cleaned Version")
    print("="*40)
    rogue = Character(name="Agile Rogue", str_val=2, dex_val=2, sta_val=2)
    simulate_combat(rogue, num_rounds=5)

SIMULATION: Final Cleaned Version
--- Agile Rogue enters combat! ---
  Base Stats: STR 2, DEX 2, STA 2
  Max AP (Permanent Cap): 6.00
  Max Endurance: 18.00
  Base AP Regen/Round: 5.00
  Endurance Regen/Round: 5.00
  Initiative Roll grants: +4 AP (One-time starting bonus)
  Starting AP (Overfilled): 10.00
--------------------

--- ROUND 1 ---
  Start of Round Status:
    AP: 10.00 / 6.00 (Current/Cap)
    Endurance: 18.00 / 18.00
  Actions Taken: 1 x 'Simple Attack' (6 AP spent)
    Net Endurance Change: -1.00 | New Endurance: 17.00 (94.4%)
    Effective AP Regen this round: 4.72

--- ROUND 2 ---
  Start of Round Status:
    AP: 6.00 / 6.00 (Current/Cap)
    Endurance: 17.00 / 18.00
  Actions Taken: 1 x 'Simple Attack' (6 AP spent)
    Net Endurance Change: -1.00 | New Endurance: 16.00 (88.9%)
    Effective AP Regen this round: 4.44

--- ROUND 3 ---
  Start of Round Status:
    AP: 4.44 / 6.00 (Current/Cap)
    Endurance: 16.00 / 18.00
  Actions Taken: 0 x 'Simple Attack' (0 AP spent)
